DeepBlink is a 2D CNN to detect spots:  

the network only works on 2d images but they support 3d by offering a non maximal suppression:  
https://github.com/BBQuercus/deepBlink/blob/master/examples/3d_prediction.ipynb  
(used below)

#### Installation

```
conda create -n deepblink python=3.7  
conda activate deepblink

## if GPUs are available install cuda and tf2_gpu according to the nvidia drive

conda install -c bbquercus deepblink

```

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
tf.config.experimental.list_physical_devices()

In [ ]:
is_gpu = len(tf.config.experimental.list_physical_devices('GPU')) > 0
is_gpu

In [ ]:
from glob import glob
import os
import pandas as pd
import tifffile as tif
import numpy as np
import time

import deepblink as pink
from skimage import io
import trackpy as tp
import subprocess

In [ ]:
## analysis type: '' (for the original analysis) OR 'adj_spots'
analysis = '_adj_spots'

In [ ]:
path = '/YOUR/PATH/'
org_ims_dir = os.path.join(path,f'simulations{analysis}')
db_dir = os.path.join(path, 'deepblink')
os.makedirs(db_dir, exist_ok=True)

In [ ]:
results_path_simul = os.path.join(
    db_dir, f'results{analysis}')
os.makedirs(results_path_simul, exist_ok=True)

### Load pretrained model
We trained our own network, but results were not as good on the simulated data.  
We also tested the pretrained network "smfish.n5", but its not as good.

In [ ]:
!deepblink download -i Particle

In [ ]:
model = pink.io.load_model("particle.h5")

### Load Images

In [ ]:
org_ims_paths = glob(os.path.join(org_ims_dir,'*','*.tif'))
len(org_ims_paths)

In [ ]:
# Those images are used for accuracy analysis
org_simul_ims_paths = [p for p in org_ims_paths 
                       if "embryos_FISH" not in p
                      and "000spots" not in p
                      and os.path.exists(f'{p[:-4]}.loc')]

len(org_simul_ims_paths)

In [ ]:
all_csvs = [f'{f[:-4]}.loc' for f in org_simul_ims_paths]

In [ ]:
# # save a list of corresponding filename and file number in the list. will be used for all analysis

# ## get imidiate dir and file name:
# all_tifs = ["/".join(f.split('/')[-2:]) for f in all_tifs]
# df_name_nums = pd.DataFrame({'name':all_tifs}) #'num': range(len(all_tifs)) ,
# df_name_nums.to_csv('ims_and_corresponding_num.csv')

## Predict 

#### In 2D

In [ ]:
## Grid search on this variable:
radius_refinement = [2,3,4]

In [ ]:
results_path_simul_2d = os.path.join(results_path_simul, '2D')
os.makedirs(results_path_simul_2d, exist_ok=True)

In [ ]:
times = [[] for i in range(len(radius_refinement))]

for i,im_path in enumerate(org_simul_ims_paths):

    im_dir = os.path.basename(os.path.dirname(im_path))
    im_name = os.path.basename(im_path[:-4])

    im_3d = tif.imread(im_path)

    for j,r in enumerate(radius_refinement):

        pad_width = r
        
        df = pd.DataFrame()
    
        time_im = 0

        for slice, image_curr in enumerate(im_3d):

            beg_time = time.time()

            # deepBlink prediction
            yx = pink.inference.predict(image=image_curr, model=model)
            y, x = yx.T.copy()

            time_im += (time.time() - beg_time)

            # pad to avoid error for spot close to the edges
            yx = yx + pad_width
            image_curr = np.pad(
                image_curr, pad_width=pad_width, mode='constant', constant_values = 0
            )

            # Refinement with trackpy
            df_curr = tp.refine_com(
                raw_image=image_curr, image=image_curr, radius=r, coords=yx
            )
            df_curr["x"] = x
            df_curr["y"] = y
            df_curr["z"] = slice
            df = df.append(df_curr, ignore_index=True)

        times[j].append(time_im)

        df.to_csv(os.path.join(results_path_simul_2d ,f'radius{r}__{im_dir}__{im_name}.csv'), index=False)


In [ ]:
# Times:
for i,r in enumerate(radius_refinement):
    pd.DataFrame({"file":org_simul_ims_paths, "times":times[i]}
                ).to_csv(os.path.join(results_path_simul_2d ,f'radius{r}_times.csv'))

## Unite spots (2D to 3D)

In [ ]:
## grid search search_range)
search_range = [1,2,3]
gap_frames = 0
min_frames = 2

In [ ]:
times = [[[] for j in range(len(search_range))] for i in range(len(radius_refinement))] 

for i,im_path in enumerate(org_simul_ims_paths):
    
    im_dir = os.path.basename(os.path.dirname(im_path))
    im_name = os.path.basename(im_path[:-4])
    
    for j,r in enumerate(radius_refinement):

        df = pd.read_csv(os.path.join(results_path_simul_2d, f'radius{r}__{im_dir}__{im_name}.csv'))
        
        if not df.empty:

            for k,s in enumerate(search_range):
            
                beg_time = time.time()

                track = tp.link(df.rename({"z": "frame"}, axis=1), 
                             search_range=s, memory=gap_frames)
                track = tp.filter_stubs(track, threshold=min_frames
                                    ).rename({"frame": "z"}, axis=1)

                # Index of brightest particles
                idx = track.groupby(["particle"])["mass"].transform(max) ==track["mass"]
                df_nms = track[idx]

                # Remove tracks from all spots
                df_without_track = df[
                    ~df.set_index(["x", "y", "z", "mass"]).index.isin(
                        track.set_index(["x", "y", "z", "mass"]).index)]

                # Add back nms (brightest spots)
                df_clean = pd.concat([df_nms, df_without_track]).reset_index(drop=True)
                df_clean.to_csv(os.path.join(results_path_simul ,f'radius{r}__search{s}__{im_dir}__{im_name}.csv'), index=False)

                times[j][k].append(time.time()-beg_time)

        else:
            times[j][k].append(np.nan)

In [ ]:
# Times:
for i,r in enumerate(radius_refinement):
    for j,s in enumerate(search_range):
        pd.DataFrame({"file":org_simul_ims_paths, "times":times[i][j]}).to_csv(os.path.join(results_path_simul ,f'radius{r}search{s}_3dunite_times.csv'))